# Development of the model class that will be used for the backend

This notebook is intended for testing and experimentation purposes only. Here the classes for the backend are written, which are used in the PROD environment.
Since minor changes were made directly in the server script, there are consequently deviations from this script here.



## Imports


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall albumentations
!pip install -qU torch_snippets

Found existing installation: albumentations 0.1.12
Uninstalling albumentations-0.1.12:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/albumentations-0.1.12.dist-info/*
    /usr/local/lib/python3.7/dist-packages/albumentations/*
Proceed (y/n)? y
  Successfully uninstalled albumentations-0.1.12
     |████████████████████████████████| 49 kB 3.0 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 78 kB 7.0 MB/s 
     |████████████████████████████████| 58 kB 6.1 MB/s 
     |████████████████████████████████| 10.9 MB 46.3 MB/s 
     |████████████████████████████████| 231 kB 49.8 MB/s 
     |████████████████████████████████| 948 kB 60.4 MB/s 
     |████████████████████████████████| 51 kB 2.3 MB/s 


In [ ]:
import torch
from torch_snippets import *
from PIL import Image

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import numpy as np

import pandas as pd

import os 
from torchvision.ops import nms

In [ ]:
model_path = '/content/drive/MyDrive/outputs/models/faster_rcnn_mobilenetv3_large.pt'
rootdir = '/content/drive/MyDrive/data'
image_path = '/content/drive/MyDrive/data/euro-coin-dataset'
df_test = pd.read_csv(os.path.join(rootdir, 'test.csv'))

## Pipeline

In [ ]:
# Parses and processes image

class Pipeline():
    def __init__(self, device, target_width = 504, target_height = 504):
        self.target_width = target_width
        self.target_height = target_height
        self.device = device

    def image_to_tensor(self, img):
        img = torch.tensor(img).permute(2,0,1)
        return img.to(self.device).float()
    
    def preprocess_image(self, img):
        img = np.array(img.resize((self.target_width, self.target_height), resample=Image.BILINEAR))/255.
        return img

    def pipe(self, image):
        """
        image: PIL.Image --> convert('RGB')
        """
        image = self.preprocess_image(image)
        image = self.image_to_tensor(image)
        return image

## Model Class

In [ ]:
ima = None
class CoinDetector():
    def __init__(self, model_path, target2label, threshold = None):
        
        self.num_classes = len(target2label)
        self.target2label = target2label
        self.threshold = threshold

        # read model
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=False)
        in_features = self.model.roi_heads.box_predictor.cls_score.in_features
        self.model.roi_heads.box_predictor = FastRCNNPredictor(in_features, self.num_classes)
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.to(self.device)

        # create pipeline
        self.pipeline = Pipeline(self.device)


        print('Finished initializing CoinDetector')

    def calculate_coin_value(labels, confs, threshold = None, size = None):
        coin_value = 0
        if threshold is None:
            threshold = 0
        for ix, label in enumerate(labels):
            if confs[ix] > threshold:
                coin_value += label
        return coin_value

    def decode_output(self, output, normalize_bbs = True, size = None):
        'convert tensors to numpy arrays'
        bbs = output['boxes'].cpu().detach().numpy().astype(np.uint16)
        labels = np.array([self.target2label[i] for i in output['labels'].cpu().detach().numpy()])
        confs = output['scores'].cpu().detach().numpy()
        ixs = nms(torch.tensor(bbs.astype(np.float32)), torch.tensor(confs), 0.05)
        bbs, confs, labels = [tensor[ixs] for tensor in [bbs, confs, labels]]
        print(type(bbs))
        if normalize_bbs and size is not None:
          print(bbs)
          bbs = bbs.astype(float)
          width, height = size
          bbs[:, [0, 2]] = bbs[:, [0, 2]] / width
          bbs[:, [1, 3]] = bbs[:, [1, 3]] / height
          print(bbs)

        if len(ixs) == 1:
            bbs, confs, labels = [np.array([tensor]) for tensor in [bbs, confs, labels]]
        return bbs.tolist(), confs.tolist(), labels.tolist()

    def predict(self, img):
        global ima
   
        img = self.pipeline.pipe(img)
        width, height = img.shape[1:]
        print(width, height)
        ima = img
        
        img = [img]

        self.model.eval()
        outputs = self.model(img)
        for ix, output in enumerate(outputs):
            bbs, confs, labels = self.decode_output(output, size=(width, height))
            print(bbs)
            
            info = [f'{l}@{c:.2f}' for l,c in zip(labels, confs)]
            print(info)
            # show(img[ix].cpu().permute(1,2,0), bbs=bbs, texts=info, sz=5 )
            coin_value = CoinDetector.calculate_coin_value(labels, confs, threshold = self.threshold)
            return coin_value

target2label = {1: 1, 2: 10, 3: 100, 4: 2, 5: 20, 6: 200, 7: 5, 8: 50, 0: 'background'}
cd = CoinDetector(model_path=model_path, target2label=target2label)

Finished initializing CoinDetector


In [ ]:
# Test case

for ix, fn in enumerate(df_test['filename'].head(200).unique()):
  if ix == 1:
    break

  try:
    data = df_test[df_test['filename'] == fn]
    folder = data['folder'].values[0]
    filepath = os.path.join(image_path, folder, fn)
    img = Image.open(filepath)
    img = img.transpose(Image.ROTATE_270)

    print(type(img)) #0-255
    print(img.size)

    value = cd.predict(img)
    print('='*10)
    print(f"Actuals:\t {data['pose'].sum()}")
    print(f'Calculated:\t {value}')
    print('='*10)
  except Exception as e:
    print(e)

<class 'PIL.Image.Image'>
(3024, 3024)
504 504
<class 'numpy.ndarray'>
[[285  46 391 156]
 [190 197 297 310]
 [346 178 455 286]
 [116  44 231 157]
 [270 314 375 411]
 [ 47 169 146 272]
 [ 85 321 197 427]]
[[0.56547619 0.09126984 0.77579365 0.30952381]
 [0.37698413 0.39087302 0.58928571 0.61507937]
 [0.68650794 0.3531746  0.90277778 0.56746032]
 [0.23015873 0.08730159 0.45833333 0.31150794]
 [0.53571429 0.62301587 0.74404762 0.81547619]
 [0.09325397 0.33531746 0.28968254 0.53968254]
 [0.16865079 0.63690476 0.39087302 0.84722222]]
[[0.5654761904761905, 0.09126984126984126, 0.7757936507936508, 0.30952380952380953], [0.376984126984127, 0.39087301587301587, 0.5892857142857143, 0.6150793650793651], [0.6865079365079365, 0.3531746031746032, 0.9027777777777778, 0.5674603174603174], [0.23015873015873015, 0.0873015873015873, 0.4583333333333333, 0.3115079365079365], [0.5357142857142857, 0.623015873015873, 0.7440476190476191, 0.8154761904761905], [0.09325396825396826, 0.3353174603174603, 0.28968253